In [1]:
import numpy as np
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# (데이터 출처: 캐글)
- 링크: [Rock Paper Scissors Dataset](https://www.kaggle.com/datasets/sanikamal/rock-paper-scissors-dataset)
- train 데이터 2,520개, test 데이터 372개
- 원본 이미지 크기는 300x300이나 연산 수행 소요시간 및 메모리 관리를 위해 150x150로 축소
- 원본 이미지는 RGBA의 4채널이나, 연산 수행 소요시간 및 메모리 관리를 위해 RGB 3채널로 축소

In [3]:
# 데이터 경로 설정
train_dir = "/content/drive/My Drive/rock_scissors_paper/train"
test_dir = "/content/drive/My Drive/rock_scissors_paper/test"

# 1. 학습 데이터에 대한 데이터 증강

- 이미지를 회전, 이동, 확대/축소 등의 방법으로 변형하여 데이터를 증강
- validation_split=0.2는 학습 데이터의 20%를 검증 데이터로 사용한다는 의미
- ImageDataGenerator는 배치 단위로 데이터를 로드하므로, 메모리 사용량을 크게 줄일 수 있음
- 노드에 제시된 load_data()를 사용하다 메모리 부족으로 진행하지 못하여, 배치 단위로 데이터를 로드함

In [4]:
# 데이터 증강 및 로딩
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 검증 데이터 분리
)

# 테스트 데이터에 대한 전처리 설정, 픽셀값을 0~1 사이로 조정(정규화 과정))
test_datagen = ImageDataGenerator(rescale=1./255)

# 학습 데이터를 로드하는 제너레이터
# 이미지 크기를 150x150으로 조정하고, 배치 크기는 32로 설정
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

# 검증 데이터를 로드하는 제너레이터
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

# 테스트 데이터를 로드하는 제너레이터
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


# 2. 모델 정의

In [ ]:
- 신경망 층을 3개로 늘림
- 과적합을 피하기 위해 Dropout()을 삽입
- MaxPooling2D()를 삽입

In [5]:
n_channel_1 = 32
n_channel_2 = 64
n_channel_3 = 128
n_dense = 512
num_classes = 3

model = Sequential([
    Conv2D(n_channel_1, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(n_channel_2, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Flatten(),
    Dense(n_dense, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# 모델 컴파일, 옵티마이저, 손실 함수, 평가 지표를 지정
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 3. 모델 학습

In [7]:
n_train_epoch = 20

# 조기 종료(Early Stopping) 콜백을 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
history = model.fit(
    train_generator,
    epochs=n_train_epoch,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 1060s 16s/step - accuracy: 0.3319 - loss: 2.2325 - val_accuracy: 0.3333 - val_loss: 1.0987
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.3170 - loss: 1.0991 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.3414 - loss: 1.0987 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.3418 - loss: 1.0986 - val_accuracy: 0.3353 - val_loss: 1.0987
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - accuracy: 0.3625 - loss: 1.0964 - val_accuracy: 0.3393 - val_loss: 1.0998
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.3434 - loss: 1.0981 - val_accuracy: 0.3353 - val_loss: 1.0987
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.3361 - loss: 1.0986 - val_accuracy: 0.3512 - val_loss: 1.0998
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.3299 - loss: 1.1004 - val_accuracy: 0.4643 - val_loss: 

# 4. 성능 평가

In [8]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

12/12 - 196s - 16s/step - accuracy: 0.9355 - loss: 0.3523
Test Loss: 0.3523
Test Accuracy: 0.9355


# (회고)
- 과적합과 과소적합의 판별은 이해하여, 이를 피하기 위한 노력은 하였음
- 이미지 데이터의 전처리, CNN, NN 등에 대한 학습이 필요함
- 이미지 데이터의 정규화 부분에서 테스트 데이터까지 정규화하여 에러가 발생하였으나 곧 해결함
- 연산 수행 시간이 과도하게 소요되어, 이에 대한 적절한 대응이 필요함을 느낌